<a href="https://colab.research.google.com/github/YCPNG/data-course-sample/blob/main/U49_cf_user_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [ ]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-06 11:56:13--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  18.8MB/s    in 0.8s    

2022-01-06 11:56:14 (18.8 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2022-01-06 11:56:14--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  14.2MB/s    in 0.7s    

202

In [ ]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
# 原始All_beauty.csv無header，此處給予header
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [ ]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
ratings.drop(columns=['unixReviewTime'],inplace = True)

In [ ]:
ratings.head(3)

,asin,reviewerID,overall,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,2014-08-10


In [ ]:
ratings['asin'].value_counts()

B000FOI48G    8672
B000GLRREU    8341
1620213982    4792
B001QY8QXM    4544
B01DKQAXC0    4246
              ... 
B0001WYM56       1
B00XDCRKAA       1
B00VEYJQNS       1
B01CGZ3YTK       1
B01GGR7V1Q       1
Name: asin, Length: 32586, dtype: int64

In [ ]:
# user有重複留評價的情形（或是資料重複）
ratings.groupby(['asin', 'reviewerID', 'overall', 'DATE']).size().sort_values(ascending = False)

asin        reviewerID      overall  DATE      
B00W259T7G  A3NFZN1GS1RKR9  5.0      2013-07-19    11
B01DLR9IDI  AUX122XW8ONG6   5.0      2018-08-23    11
            A11QGZ39A7ZF0X  5.0      2018-08-20    10
B007V6JNE0  AKJHHD5VEH7VG   3.0      2008-05-25     9
B00W259T7G  A1MHFSMKSOED47  5.0      2014-10-08     5
                                                   ..
B00SQ3S6TA  A3509WLF7GJT4W  4.0      2017-06-11     1
            A353IAF5Y1XE53  2.0      2017-02-28     1
            A359PIS419ON4P  5.0      2015-12-31     1
            A35H7GVUWLIKUQ  5.0      2016-06-18     1
0061073717  A2XMFX1BR0IJFJ  5.0      2000-06-03     1
Length: 362275, dtype: int64

In [ ]:
# 清除ratings重複資料
ratings.drop_duplicates(inplace = True)

In [ ]:
ratings.reset_index(inplace = True)

In [ ]:
ratings = ratings.drop('index', axis = 1)

In [ ]:
ratings.head(3)

,asin,reviewerID,overall,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,2014-08-10


In [ ]:
# 只保留｀overall` > 3.0的評價
ratings_all = ratings
ratings = ratings[ratings['overall'] > 3.0]
ratings

,asin,reviewerID,overall,DATE
1,0143026860,A2F5GHSXFQ0W6J,4.0,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,2011-12-25
5,0143026860,A36NF437WZLQ9E,5.0,2010-02-26
...,...,...,...,...
362264,B01HJ84SGM,A7WHO9ZFPHU2B,5.0,2017-05-09
362268,B01HJEGTYK,A338U2R53FLF9U,5.0,2017-10-28
362269,B01HJEGTYK,A3EN87RJFCIORB,4.0,2017-07-22
362271,B01HJEGTYK,A3FSOR5IJOFIBE,5.0,2017-03-16


In [ ]:
metadata_clean = metadata.drop_duplicates('asin')
metadata_clean.reset_index(inplace = True)

## 資料切分

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [ ]:
#產生舊戶名單
old_users = list(set(users) & set(ratings_trainings['reviewerID']))
#產生新戶名單
new_users = list(set(users) - set(ratings_trainings['reviewerID']))
len(old_users), len(new_users)

(22, 400)

# **Rule-based recommend** (from week 1)

In [ ]:
# 6個月內的資料
ratings_testings_6m = ratings[
    (ratings['DATE'] >= '2018-03-01') & 
    (ratings['DATE'] <= '2018-09-01')
    ]

# 產生6個月內的rule-based推薦清單
ratings_testings_6m_lst = ratings_testings_6m.groupby(by=['asin']).size().sort_values(ascending=False).index.tolist()[:10]

## 產生推薦

In [ ]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

# header: user_id,item_id,rating,timestamp

def recommender(training_data, users=[], k=10):

    # loading data from dataframe
    # user_to_items dict:
    # {
    #   'user': {
    #       'item': ratings...
    #   }...
    # }
    user_to_items = defaultdict(dict)  # default值以一個dict()方法產生
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])

        user_to_items[user][item] = rating

    print("total users before filtering: ", len(user_to_items))
    print("user_to_items: ", dict(list(user_to_items.items())[0:5]))

    # remove obscure user to decrease data size
    # filtering params
    remove_obscure_user = True
    user_rating_threshold = 3
    all_users = list(user_to_items.keys())
    for user in all_users:
        ratings = user_to_items[user]
        if remove_obscure_user and len(ratings) < user_rating_threshold:
            del user_to_items[user]

    print("total users after filtering: ", len(user_to_items))

    # generate item to user mapping dict
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for user, items in user_to_items.items():
        for item, rating in items.items():
            item_to_users[item][user] = rating

    # prepare data of computing user similarity 
    init_sim = lambda: [0 for _ in range(3)]
    factory = lambda: defaultdict(init_sim)
    pre_user_similarity = defaultdict(factory)
    n = len(item_to_users)
    index = 0
    for item, user_ratings in item_to_users.items():
        if len(user_ratings) > 1:
            # print(f"item: {item} have been rated by {len(user_ratings)} users progress: {index}/{n}")
            for user1, user2 in combinations(user_ratings.keys(), 2):
                xy = user_ratings[user1] * user_ratings[user2]
                xx = user_ratings[user1] ** 2
                yy = user_ratings[user2] ** 2
                pre_user_similarity[user1][user2][0] += xy
                pre_user_similarity[user1][user2][1] += xx
                pre_user_similarity[user1][user2][2] += yy

                pre_user_similarity[user2][user1][0] += xy
                pre_user_similarity[user2][user1][1] += xx
                pre_user_similarity[user2][user1][2] += yy
        index += 1

    user_similarity = {}
    for src_user in pre_user_similarity:
        user_similarity_order = []
        for dst_user, val in pre_user_similarity[src_user].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(user_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    user_similarity_order.insert(i, (dst_user, similarity))
                    break
            else:
                user_similarity_order.append((dst_user, similarity))
        user_similarity[src_user] = user_similarity_order

    recommendation = {}
    for user in users:
        if user in user_similarity:
            sim_users = user_similarity[user]
            recommended_items = []
            recommended_items_set = set()
            user_have_rated = set(user_to_items[user])
            stop_recommend = False
            for sim_user, _ in sim_users:
                items_from_sim_user = sorted(list(user_to_items[sim_user].items()), key=lambda item: item[1])
                for item, _ in items_from_sim_user:
                    if item not in user_have_rated and item not in recommended_items_set:
                        recommended_items.append(item)
                        recommended_items_set.add(item)
                    if len(recommended_items) >= k:
                        stop_recommend = True
                        break
                if stop_recommend:
                    break
            recommendation[user] = recommended_items
        else:
            # recommendation[user] = []
            recommendation = {user : ratings_testings_6m_lst for user in users}
     
    

    return recommendation


ratings_by_user = recommender(ratings_trainings, users)


total users before filtering:  245390
user_to_items:  {'A2F5GHSXFQ0W6J': {'0143026860': 4.0}, 'A1572GUYS7DGSR': {'0143026860': 4.0}, 'A1PSGLFK1NSVO': {'0143026860': 5.0}, 'A6IKXKZMTKGSC': {'0143026860': 5.0}, 'A36NF437WZLQ9E': {'0143026860': 5.0}}
total users after filtering:  3611


In [ ]:
ratings_by_user

## 結果評估

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

0.10588235294117647